#Capstone Project Notebook


In [1]:
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [ ]:
!pip  install urllib2

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


In [ ]:
#https://api.foursquare.com/v2/
#client ID
#client secret 
#version

##Scraping data from Wikipedia



In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
http = urllib3.PoolManager()
page = http.request('GET',url)
soup = BeautifulSoup(page.data, 'html.parser')


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
table_data = []
for tr in soup.find_all('tr')[2:]:
  tds = tr.find_all('td')
  try:
    table_data.append([(tds[0].text)[:-1],(tds[1].text)[:-1],(tds[2].text)[:-1]])
  except:
    print("index out of range")
  

index out of range


##Dataframe From Scrapped Data


In [4]:
#table_data[0:5]
df = pd.DataFrame(table_data,columns = ['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


##removing Cells which contains "Not Assigned" in Brorough Column


In [5]:
df.drop(df[df['Borough'] == 'Not assigned'].index,inplace=True)

In [6]:
df.head()

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##Grouping the neighbourhood

In [7]:
df_neighbourhood = df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: " ,".join(x))

df_neighbourhood.drop([0,1],inplace = True)
df_neighbourhood.head()

,PostalCode,Borough,Neighborhood
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn
6,M1H,Scarborough,Cedarbrae


In [8]:
for index, row in df_neighbourhood.iterrows():
  if row["Neighborhood"] == "Not assigned":
    row["Neighbourhood"] = row["Borough"]
df_neighbourhood.head()

,PostalCode,Borough,Neighborhood
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn
6,M1H,Scarborough,Cedarbrae


In [ ]:
df_neighbourhood.shape

(104, 3)

##Mapping longitude and lattitude with all the neighbourhood

In [9]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.rename(columns={"Postal Code":"PostalCode"},inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_neighbourhood_new = df_neighbourhood.merge(coordinates,on = "PostalCode",how='left')
df_neighbourhood_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_neighbourhood_new[df_neighbourhood_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [12]:
import folium
from sklearn.cluster import KMeans

In [23]:
df_neighbourhood_new.groupby('Borough').count()["Neighborhood"]


Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
NS                   1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [24]:
df_toronto = df_neighbourhood_new[df_neighbourhood_new['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [25]:
df_toronto.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64

In [42]:
boroughs = df_toronto['Borough'].unique().tolist()
boroughs

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [43]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [35]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print(lat_toronto,lon_toronto)

43.66713498717948 -79.38987324871795


In [50]:
toronto_map = folium.Map(location=[lat_toronto, lon_toronto],zoom_start= 12)
for lon,lat,borough,neighboor in zip(df_toronto['Longitude'],
                                     df_toronto['Latitude'],
                                     df_toronto['Borough'],
                                     df_toronto['Neighborhood']):
  label_text = borough + '_' + neighboor
  label = folium.Popup(label_text)
  folium.CircleMarker(
      [lat,lon],
      radius = 5,
      popup = label,
      color=borough_color[borough],
      fill_color=borough_color[borough],
      fill_opacity=0.7).add_to(toronto_map)

  
toronto_map

#Getting Foursquare Data


In [66]:
Client_id = 'C4NP4HVGSUILYLOCOSHM0BTRWDEO32LIFPK0KQXLY4DEX3ZS'
Client_secret = '1HDARRNNSMA3VFSGW1JTLONGACMXDR5DICQKF3DKWZTPZEMD'
version =20170630
limit = 100
radius = 500

In [72]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_id, 
            Client_secret, 
            version, 
            lat, 
            lon, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [75]:
toronto_venues.shape

(370, 7)

In [76]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Monarch Park Stadium,43.678144,-79.324038,Soccer Field
1,The Beaches,43.676357,-79.293031,Lake Inez,43.672520,-79.320712,Asian Restaurant
2,The Beaches,43.676357,-79.293031,Udupi Palace,43.672480,-79.321275,Indian Restaurant
3,The Beaches,43.676357,-79.293031,Monarch Park,43.678719,-79.325957,Park
4,The Beaches,43.676357,-79.293031,Godspeed Brewery,43.672620,-79.319228,Brewery


In [81]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,3,3,3,3,3,3
"Brockton, Parkdale Village, Exhibition Place",3,3,3,3,3,3
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
Central Bay Street,1,1,1,1,1,1
Christie,29,29,29,29,29,29
Church and Wellesley,21,21,21,21,21,21
"Commerce Court, Victoria Hotel",3,3,3,3,3,3
Davisville North,4,4,4,4,4,4
"Dufferin, Dovercourt Village",32,32,32,32,32,32


In [82]:
toronto_venues['Venue Category'].unique()

array(['Soccer Field', 'Asian Restaurant', 'Indian Restaurant', 'Park',
       'Brewery', 'Café', 'Bar', 'Snack Place', 'Grocery Store',
       'Indian Chinese Restaurant', 'Sandwich Place', 'Baseball Field',
       'Diner', 'Gym', 'Dog Run', 'Discount Store', 'Convenience Store',
       'Pet Store', "Women's Store", 'Shopping Plaza', 'Cosmetics Shop',
       'Burger Joint', 'Bowling Alley', 'Pizza Place', 'Breakfast Spot',
       'Pharmacy', 'Farmers Market', 'Pakistani Restaurant',
       'Fish & Chips Shop', 'Sushi Restaurant', 'Fast Food Restaurant',
       'Steakhouse', 'Burrito Place', 'Restaurant', 'Light Rail Station',
       'Auto Workshop', 'Music Venue', 'Coffee Shop', 'Food Truck',
       'French Restaurant', 'Playground', 'Skating Rink', 'Office',
       'Gastropub', 'Cocktail Bar', 'Bank', 'Bus Station',
       'Salon / Barbershop', 'Comic Shop', 'Garden Center', 'Yoga Studio',
       'Butcher', 'Hot Dog Joint', 'Boat or Ferry', 'Harbor / Marina',
       'Scenic Lookout',

##Analyze each neighborhood

In [86]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']])
to_onehot['Neighborhood'] = toronto_venues['Neighborhood']
to_onehot.head()

,Venue Category_Asian Restaurant,Venue Category_Auto Workshop,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Boat or Ferry,Venue Category_Bowling Alley,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Butcher,Venue Category_Café,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_Comic Shop,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Dog Run,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Food Truck,Venue Category_French Restaurant,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gastropub,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Venue Category_Harbor / Marina,Venue Category_Hot Dog Joint,Venue Category_Indian Chinese Restaurant,Venue Category_Indian Restaurant,Venue Category_Indie Theater,Venue Category_Light Rail Station,Venue Category_Music Venue,Venue Category_Office,Venue Category_Pakistani Restaurant,Venue Category_Park,Venue Category_Pet Store,Venue Category_Pharmacy,Venue Category_Pizza Place,Venue Category_Playground,Venue Category_Recording Studio,Venue Category_Restaurant,Venue Category_Salon / Barbershop,Venue Category_Sandwich Place,Venue Category_Scenic Lookout,Venue Category_Shopping Plaza,Venue Category_Skate Park,Venue Category_Skating Rink,Venue Category_Snack Place,Venue Category_Soccer Field,Venue Category_Steakhouse,Venue Category_Sushi Restaurant,Venue Category_Theater,Venue Category_Tibetan Restaurant,Venue Category_Women's Store,Venue Category_Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,The Beaches
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches


In [99]:
to_grouped = to_onehot.groupby(['Neighborhood']).mean().reset_index()
to_grouped.head()

,Neighborhood,Venue Category_Asian Restaurant,Venue Category_Auto Workshop,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Boat or Ferry,Venue Category_Bowling Alley,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Butcher,Venue Category_Café,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_Comic Shop,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Dog Run,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Food Truck,Venue Category_French Restaurant,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gastropub,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Venue Category_Harbor / Marina,Venue Category_Hot Dog Joint,Venue Category_Indian Chinese Restaurant,Venue Category_Indian Restaurant,Venue Category_Indie Theater,Venue Category_Light Rail Station,Venue Category_Music Venue,Venue Category_Office,Venue Category_Pakistani Restaurant,Venue Category_Park,Venue Category_Pet Store,Venue Category_Pharmacy,Venue Category_Pizza Place,Venue Category_Playground,Venue Category_Recording Studio,Venue Category_Restaurant,Venue Category_Salon / Barbershop,Venue Category_Sandwich Place,Venue Category_Scenic Lookout,Venue Category_Shopping Plaza,Venue Category_Skate Park,Venue Category_Skating Rink,Venue Category_Snack Place,Venue Category_Soccer Field,Venue Category_Steakhouse,Venue Category_Sushi Restaurant,Venue Category_Theater,Venue Category_Tibetan Restaurant,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Berczy Park,0.000000,0.000000,0.0,0.000000,0.0,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.058824,0.0,0.000000,0.0,0.000000,0.0,0.0,0.058824,0.000000,0.058824,0.0,0.0,0.058824,0.000000,0.0,0.0,0.058824,0.0,0.000000,0.0,0.000000,0.0,0.0,0.058824,0.058824,0.000000,0.0,0.0,0.058824,0.058824,0.0,0.000000,0.000000,0.058824,0.000000,0.0,0.000000,0.000000,0.000000,0.058824,0.0,0.0,0.000000,0.058824,0.0,0.0,0.058824,0.0,0.058824,0.058824,0.0,0.000000,0.000000,0.0,0.058824,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.058824
3,Central Bay Street,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Christie,0.034483,0.000000,0.0,0.034483,0.0,0.000000,0.0,0.0,0.034483,0.034483,0.000000,0.0,0.0,0.000000,0.103448,0.0,0.0,0.000000,0.0,0.034483,0.0,0.034483,0.0,0.0,0.000000,0.034483,0.034483,0.0,0.0,0.000000,0.000000,0.0,0.103448,0.034483,0.000000,0

In [88]:
to_grouped.shape

(37, 66)

In [103]:
pizza_places =to_grouped[["Neighborhood","Venue Category_Pizza Place"]]
pizza_places.head()

,Neighborhood,Venue Category_Pizza Place
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.058824
3,Central Bay Street,0.000000
4,Christie,0.000000


In [109]:
to_clustering = pizza_places.drop(["Neighborhood"], 1)
to_clustering.head()

,Venue Category_Pizza Place
0,0.000000
1,0.000000
2,0.058824
3,0.000000
4,0.000000


In [110]:
from sklearn.cluster import KMeans
k = 3
kmean = KMeans(n_clusters=k,random_state=0)
kmean.fit(to_clustering)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [111]:
kmean.labels_

array([0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1], dtype=int32)

In [114]:
pizza_places['Cluster labels'] = kmean.labels_
pizza_places.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Neighborhood,Venue Category_Pizza Place,Cluster labels
0,Berczy Park,0.000000,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.058824,1
3,Central Bay Street,0.000000,0
4,Christie,0.000000,0


In [119]:
pizza_places = pizza_places.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")
pizza_places.head()

,Neighborhood,Venue Category_Pizza Place,Cluster labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,Induna,43.644566,-79.323354,Boat or Ferry
0,Berczy Park,0.0,0,43.644771,-79.373306,Andante || - Ian's Boat,43.641694,-79.323752,Boat or Ferry
0,Berczy Park,0.0,0,43.644771,-79.373306,Toronto Outer Harbour Marina,43.642815,-79.326825,Harbor / Marina
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0,43.636847,-79.428191,Tommy Thompson Park,43.634180,-79.323671,Park
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0,43.636847,-79.428191,leslie spits,43.637796,-79.322931,Scenic Lookout


In [121]:
pizza_places.sort_values(['Cluster labels'],inplace=True)
pizza_places.head()

,Neighborhood,Venue Category_Pizza Place,Cluster labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,Induna,43.644566,-79.323354,Boat or Ferry
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Gerrard India Bazaar,43.672086,-79.323403,Shopping Plaza
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Toronto Cash & Carry,43.671954,-79.323412,Grocery Store
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Coxwell Variety,43.673125,-79.319248,Convenience Store
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Dollarama,43.674006,-79.319466,Discount Store


In [125]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [129]:
toronto_map = folium.Map(location=[lat_toronto, lon_toronto],zoom_start= 12)
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
marker_color = []
for lat,lon,poi,cluster in zip(pizza_places['Neighborhood Latitude'],
                               pizza_places['Neighborhood Longitude'],
                               pizza_places['Neighborhood'],
                               pizza_places['Cluster labels']):
  label = folium.Popup(str(poi) + '-cluster'+str(cluster))
  folium.CircleMarker(
      [lat,lon],
      radius = 5,
      popup = label,
      color=rainbow[cluster-1],
      fill_color=rainbow[cluster-1],
      fill_opacity=0.7).add_to(toronto_map)
toronto_map


In [132]:
pizza_places.loc[pizza_places['Cluster labels'] == 0]

,Neighborhood,Venue Category_Pizza Place,Cluster labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,Induna,43.644566,-79.323354,Boat or Ferry
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Gerrard India Bazaar,43.672086,-79.323403,Shopping Plaza
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Toronto Cash & Carry,43.671954,-79.323412,Grocery Store
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Coxwell Variety,43.673125,-79.319248,Convenience Store
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Dollarama,43.674006,-79.319466,Discount Store
...,...,...,...,...,...,...,...,...,...
18,"Moore Park, Summerhill East",0.0,0,43.689574,-79.383160,Subway,43.690023,-79.325054,Sandwich Place
15,"Kensington Market, Chinatown, Grange Park",0.0,0,43.653206,-79.400049,Leslie St. Spit hotdog stand,43.652255,-79.322791,Hot Dog Joint
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Bombay Chowpatty,43.671875,-79.324114,Indian Restaurant
32,"The Annex, North Midtown, Yorkville",0.0,0,43.672710,-79.405678,Eulalie's Corner Store,43.672535,-79.321896,Bar


In [133]:
pizza_places.loc[pizza_places['Cluster labels'] == 1]

,Neighborhood,Venue Category_Pizza Place,Cluster labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8,"Dufferin, Dovercourt Village",0.031250,1,43.669005,-79.442259,Subway,43.666052,-79.316933,Sandwich Place
8,"Dufferin, Dovercourt Village",0.031250,1,43.669005,-79.442259,Coxwell Variety,43.673125,-79.319248,Convenience Store
8,"Dufferin, Dovercourt Village",0.031250,1,43.669005,-79.442259,Kohinoor Foods,43.672446,-79.321775,Grocery Store
8,"Dufferin, Dovercourt Village",0.031250,1,43.669005,-79.442259,Pizza Nova,43.666462,-79.316485,Pizza Place
8,"Dufferin, Dovercourt Village",0.031250,1,43.669005,-79.442259,Jonathan Ashbridge Park,43.664702,-79.319898,Park
...,...,...,...,...,...,...,...,...,...
31,"Summerhill West, Rathnelly, South Hill, Forest...",0.055556,1,43.686412,-79.400049,Seb's Cappuccino,43.684975,-79.316104,Café
2,"Business reply mail Processing Centre, South C...",0.058824,1,43.662744,-79.321558,Ashbridges Bay Skatepark,43.662548,-79.315631,Skate Park
2,"Business reply mail Processing Centre, South C...",0.058824,1,43.662744,-79.321558,Chick-n-Joy,43.665181,-79.321403,Fast Food Restaurant
2,"Business reply mail Processing Centre, South C...",0.058824,1,43.662744,-79.321558,Queen Margherita Pizza,43.664685,-79.324164,Pizza Place


In [134]:
pizza_places.loc[pizza_places['Cluster labels'] == 2]

,Neighborhood,Venue Category_Pizza Place,Cluster labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,Maple Leaf Forever Park,43.665555,-79.327087,Park
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,O Sushi,43.666684,-79.316614,Sushi Restaurant
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,The Burger's Priest,43.666731,-79.315556,Fast Food Restaurant
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,British Style Fish & Chips,43.668723,-79.317139,Fish & Chips Shop
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,The Green Wood,43.664728,-79.324117,Restaurant
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,Chick-n-Joy,43.665181,-79.321403,Fast Food Restaurant
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,Queen Margherita Pizza,43.664685,-79.324164,Pizza Place
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,Chino Locos,43.664653,-79.325584,Burrito Place
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,TTC Streetcar Loop,43.666452,-79.316455,Light Rail Station
5,Church and Wellesley,0.095238,2,43.665860,-79.383160,The Sidekick,43.664484,-79.325162,Comic Shop


##Observations

Most of Pizza restaurants are in Cluster 2 which is around Church and Wellesley, The Danforth West, Riverdale, The Danforth West, Riverdaleareas and lowest (close to zero) in Cluster 1 areas which are North Toronto West and Parkdale areas. Also, there are good opportunities to open near Chinatown, St James town as the competition seems to be low. Looking at nearby venues, it seems Cluster 1 might be a good location as there are not a lot of Asian restaurants in these areas. Therefore, this project recommends the entrepreneur to open an authentic Burmese restaurant in these locations with little to no competition. Nonetheless, if the food is authentic, affordable and good taste, I am confident that it will have great following everywhere =)